In [4]:
from reader import ebook
from main import read_list,read_json
from typing import Optional,Dict,List
import requests
from dotenv import load_dotenv
import json
import os
from pydantic import BaseModel, conint
from dataclasses import dataclass

In [5]:
load_dotenv()
API = os.getenv("HF_API")

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"



___

## Get **Summary**, **Characters** AND **Places** 

In [6]:
sum_role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and Ready to parse. nothing before or after the json file)
Input:  
1.Current Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
4.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize and explain the chapter in detail, integrating context and key developments from previous chapters and create a self containing summary and explaination. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Detailed Summary and explination of the current chapter in context of previous chapters. Use previus chapter summary as context",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}
'''

In [12]:


class SummarySchema(BaseModel):
    summary: str
    characters: Dict[str,str]
    places: Dict[str,str]

def scene_msg(text: str, context: str, characters: dict = {}, places: dict = {}) -> list:
    message = [
        {
            "role": "system",
            "content": sum_role
        },
        {
            "role": "user",
            "content": json.dumps({
                "past_context": context,
                "Current_Chapter": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

def get_summ(messages:Dict[str,str]) -> Optional[str]:
    data = {
        "messages": messages,
        "temperature": 0.7,
        "stream": False,
        "max_tokens":10000,
        "parameters": {
        "repetition_penalty": 1.3,
        "grammar": {
            "type": "json",
            "value": SummarySchema.model_json_schema()
                }
                    }
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None



In [13]:
book=ebook("./books/stranger.pdf")
title=book.get_metadata()['title']
chapters = book.get_chapters()
_,text=zip(*chapters[:5]) 

In [14]:
output_dict={
    0:
        {
            "summary":"This is the first chapter",
            "characters":{},
            "places": {}
        }
            }

In [15]:
for idx,i in enumerate(text):
    
    context=output_dict[idx]["summary"]
    characters=output_dict[idx]["characters"]
    places=output_dict[idx]["places"]
    
    mes=scene_msg(i,context,characters,places)
    
    summary_characters=get_summ(mes)
    output_json=read_json(summary_characters)
    output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")

chapter: 0 done
chapter: 1 done
chapter: 2 done
chapter: 3 done
chapter: 4 done


In [16]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
Protagonist: A middle-aged man, dressed in regular attire, seemingly calm but with a hint of sorrow. His actions show a sense of duty towards attending his mother's funeral.

Places:
Algiers: A city where the protagonist lives. The weather is described as hot. The atmosphere is busy, as indicated by the lunch at Celeste's restaurant.
Marengo: A village fifty miles from Algiers. The Home for Aged Persons is located here. The environment is not described in detail, but the distance from the village to the Home is mentioned.

Summary:
In this chapter, the protagonist learns about the death of his mother. He receives a telegram informing him of her passing and plans to attend the funeral the next day. He takes a bus to Marengo, a village fifty miles from Algiers, where the Home for Aged Persons is located. On the journey, he feels drowsy and sleeps most of the way. Upon arrival, he requests to see his mother immediately but is denied. The chapter ends with the prot

___

## Get **Scenes**

In [17]:
class SceneSchema(BaseModel):
    scenes: Dict[str,str]
print(SceneSchema.model_json_schema())

{'properties': {'scenes': {'additionalProperties': {'type': 'string'}, 'title': 'Scenes', 'type': 'object'}}, 'required': ['scenes'], 'title': 'SceneSchema', 'type': 'object'}


In [18]:
def scene_msg(text: str) -> list:
    message = [
        {
            "role": "system",
            "content": ''' IMPORTANT-> ONLY OUTPUT IN JSON.
                You are a text-to-image prompt generator. 
                Your task is to analyze the provided input text and identify distinct scenes Where Place or Time cahnges.
                For each scene, create a detailed prompt suitable for generating an image.
                Call characters by their respacted names.
                Call Places by their respected name
                
                1. **Input:**
                    - `text`: A block of narrative text.```
                2. **Output:**
                    - [prompt,prompt,prompt.....]

                ### Instructions:
                - Identify key changes in location, characters, or significant actions to define separate scenes.
                - Use descriptive language to paint a vivid picture of each scene in the prompt.
                
                   ''',
        },
        
        {
            "role": "user",
            "content": f"""TEXT: {text}""",  # This should be your input text that describes the scenes
        },]
        
    return message

def get_scene(text: str) -> Optional[str]:
    messages = scene_msg(text)
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
        "repetition_penalty": 1.3,
        "grammar": {
            "type": "json",
            "value": SceneSchema.model_json_schema()
                }
        }
    
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None

In [19]:
outputs=[]
for i in text:
    inputs=i.replace("\n"," ")
    scenes=get_scene(inputs)
    scene_json_output=read_json(scenes)
    outputs.append(scene_json_output)

___


## Get **Style**

In [21]:
def basic_llm_req(text: str) -> Optional[str]:
    messages = [
        {
            "role":"system",
            "content":"DO As Asked in The Input"
        },
        {
            "role":"user",
            "content":text
        }
    ]
    
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
    }
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None
 

In [22]:
combined_summary=""
for key,val in output_dict.items():
    val_content=val["summary"].replace("\n"," ")
    if key==0:
        continue
    output_string= f'''{combined_summary}... Chapter{key}: {val_content}'''

In [23]:
style_prompt='''Prompt:

"Analyze the following story and provide a list of image style tags that would best suit its themes, settings, and overall mood. 
The response should include the style, period, type of art, color palette etc. 4 tags per entry.dont explain it just give tags.

Response Format:

Style: Realism, Impressionism, Surrealism,etc.
Period:Romantic, Modern, Contemporary,etc.
Type:Landscape, Portrait, Abstract,etc.
Color Palette:Warm tones, Cool colors, Monochromatic,etc
Mood:Serene, Dramatic, Melancholic,etc.

Story
'''

style=basic_llm_req(f'''{style_prompt}:{combined_summary} ''')

In [24]:
style

'Title: "The Forgotten Whispers of Time"\n\n1. Style: Romantic Realism\n   Period: Victorian\n   Type: Landscape\n   Color Palette: Warm Tones (Burnt Umber, Sienna, Ochre, Sepia)\n   Mood: Melancholic\n\n2. Style: Impressionism\n   Period: Modern\n   Type: Portrait\n   Color Palette: Cool Colors (Cerulean, Cyan, Lavender, White)\n   Mood: Serene\n\n3. Style: Surrealism\n   Period: Contemporary\n   Type: Abstract\n   Color Palette: Monochromatic (Black, White, Grey)\n   Mood: Dramatic\n\n4. Style: Expressionism\n   Period: Early 20th Century\n   Type: Landscape\n   Color Palette: Vibrant (Crimson, Lime, Cobalt, Ultramarine)\n   Mood: Emotional, Intense'

___

## Get **Image**

In [25]:
from huggingface_hub import InferenceClient
import time

client = InferenceClient(
    "stabilityai/stable-diffusion-3.5-large-turbo",
    token=API,
)

In [26]:
def get_images(key, text,characters,places,style):
    image = client.text_to_image(
        f"Generate a illustration for a book given the prompt,use provided character place descriptions if needed. Prompt: {text}, characters: {characters}, places:{places}// style:{style} ",
        height=528,
        width=720,
    )
    image.save(f"./output/{title}_{key}.png")

In [ ]:
for idx,i in enumerate(outputs):
    chars=output_dict[idx+1]["characters"]
    places=output_dict[idx+1]["places"]
    for jdx,j in enumerate(i):
        get_images(key=f"chapter{idx}_scene{jdx+1}",text=j,characters=chars,places=places,style=style)
        print(f"chapter{idx}_scene{jdx+1} saved")
        time.sleep(30)

chapter0_scene1 saved
chapter0_scene2 saved
chapter0_scene3 saved
chapter0_scene4 saved
